In [1]:
data_var = '2023-12-25'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6557,2023-12-25,Eua Nba,16:30,Denver Nuggets,Golden State Warriors,1.42,3.19,232.5,1.86,2.07,-6.5,1.97,1.97,Qes0xnTb,0.704225,0.313480,0.537634,0.483092,0.017705,0.6,0.8,NaN,NaN,172.398,22.700183,0.131673,221.520,98.183213,0.443225,168.248,267.758,189.80,192.34,0.0,0.0,0.0,0.0,0.542984,0.075569,0.000000,-0.99,-0.198,-2.121212,0.719476,0.8,0.080524,-3.37,-0.674,-3.249258,0.481878,0.2,-0.281878
6558,2023-12-25,Eua Nba,19:00,Los Angeles Lakers,Boston Celtics,2.31,1.72,233.5,1.86,2.07,2.5,1.97,1.97,8KW4y6r5,0.432900,0.581395,0.537634,0.483092,0.014296,0.0,0.6,NaN,NaN,228.912,90.222138,0.394135,228.960,80.397247,0.351141,178.484,215.740,167.86,272.60,0.0,0.0,0.0,0.0,0.207044,0.075569,0.000000,-0.63,-0.126,-10.396825,0.651755,0.7,0.048245,0.54,0.108,6.666667,0.618349,0.7,0.081651
6559,2023-12-25,Eua Nba,22:00,Miami Heat,Philadelphia 76ers,1.75,2.25,224.5,1.80,1.91,-4.5,2.10,1.67,nBV8zQcB,0.571429,0.444444,0.555556,0.523560,0.015873,0.6,0.8,NaN,NaN,207.512,61.177070,0.294812,150.796,19.119997,0.126794,185.996,219.584,231.12,175.50,0.0,0.0,0.0,0.0,0.176777,0.041931,0.161303,-2.16,-0.432,-1.736111,0.608833,0.5,-0.108833,-1.28,-0.256,-4.882812,0.612397,0.7,0.087603
6560,2023-12-25,Austrália Nbl,03:30,Tasmania JackJumpers,South East Melbourne,1.33,3.38,177.5,1.89,1.89,-8.5,2.01,1.71,IPnqMMw6,0.751880,0.295858,0.529101,0.529101,0.047738,0.0,0.0,NaN,NaN,159.224,34.573960,0.217140,176.256,78.079743,0.442991,0.000,0.000,133.66,305.76,0.0,0.0,0.0,0.0,0.615528,0.000000,0.114049,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6561,2023-12-25,Austrália Nbl,05:30,Sydney Kings,Illawarra Hawks,1.22,4.39,181.5,1.90,1.89,-11.5,2.08,1.67,dGomLtgC,0.819672,0.227790,0.526316,0.529101,0.047463,0.0,0.2,NaN,NaN,177.960,37.713854,0.211923,243.354,27.375954,0.112494,0.000,273.618,132.54,229.00,0.0,0.0,0.0,0.0,0.799119,0.003731,0.154621,0.00,0.000,inf,0.000000,0.0,0.000000,0.47,0.094,36.063830,0.000000,0.0,0.000000
6562,2023-12-25,Bahrein Premier League,13:30,Al Muharraq,Al Ittihad,1.57,2.31,178.5,1.83,1.83,-5.5,2.01,1.72,bLsbE9W0,0.636943,0.432900,0.546448,0.546448,0.069843,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,158.79,3213.00,0.0,1.0,0.0,0.0,0.269721,0.000000,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6563,2023-12-25,Bósnia E Herzegovina Prvenstvo Bih,15:00,Mrkonjic Grad,Borac Banja Luka,2.59,1.46,145.5,1.88,1.79,3.5,2.08,1.66,QJViUGgd,0.386100,0.684932,0.531915,0.558659,0.071032,0.0,0.0,NaN,NaN,247.004,213.072693,0.862629,119.046,45.664550,0.383587,0.000,0.000,119.07,95.94,0.0,0.0,0.0,0.0,0.394583,0.034681,0.158815,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6564,2023-12-25,China Cba,08:35,Beijing Royal Fighters,Ningbo Rockets,1.18,4.65,205.5,1.87,1.87,-13.5,2.09,1.63,xGtoGlco,0.847458,0.215054,0.534759,0.534759,0.062511,0.6,0.4,NaN,NaN,217.756,132.408328,0.608058,445.768,190.378651,0.427080,160.082,493.144,136.96,252.72,0.0,0.0,0.0,0.0,0.841736,0.000000,0.174876,-2.79,-0.558,-0.322581,0.655250,0.4,-0.255250,1.74,0.348,10.488506,0.169151,0.2,0.030849
6565,2023-12-25,China Cba,08:35,Guangdong,Shenzhen,1.32,3.27,210.5,1.80,1.94,-9.5,2.09,1.63,YsG47dgo,0.757576,0.305810,0.555556,0.515464,0.063386,0.6,0.8,NaN,NaN,150.092,23.776719,0.158414,148.416,19.606006,0.132102,129.292,200.862,137.16,126.36,0.0,0.0,0.0,0.0,0.600810,0.052938,0.174876,0.71,0.142,2.253521,0.000000,0.0,0.000000,-0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,16:30,Eua Nba,Denver Nuggets,Golden State Warriors,232.5,1.86,1.0,Over
1,19:00,Eua Nba,Los Angeles Lakers,Boston Celtics,233.5,1.86,1.0,Over
